In [1]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np
import geopandas as gpd
%matplotlib inline

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option('precision', 5)
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

import urbanFormPy as up

from shapely.geometry import Point, LineString, MultiLineString
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, mapping, MultiLineString
from shapely.ops import cascaded_union, linemerge, nearest_points, polygonize_full

In [3]:
#initialise path, names, etc.
# use complete extension of the city to extract regions!

city_name = 'London'
place = 'Greater London, UK'
epsg = 27700

crs = {'init': 'epsg:27700', 'no_defs': True}

# city_name = 'London'
# place = 'Greater London, UK'
# epsg = 27581

# crs = {'init': 'epsg:27581', 'no_defs': True}

### Downloading

Choose between the following methods:
* *OSMplace*, provide an OSM place name (e.g. City).
* *OSMpolygon*, provide an OSM polygon (relation) name.
* *distance_from_address*, provide a precise address and define parameter "distance" (which is otherwise not necessary)

In [4]:
download_method_graph = 'OSMplace'
distance = None

In [ ]:
nodes_graph, edges_graph = up.get_network_fromOSM(download_method_graph, place, 'walk', epsg, distance = distance)

In [8]:
nodes_graph.to_file("Greater_London_nodes.shp")
edges_graph.to_file("Greater_London_edges.shp")

In [2]:
nodes_graph = gpd.read_file("Greater_London_nodes.shp")
edges_graph = gpd.read_file("Greater_London_edges.shp")

In [3]:
"""
- "same_uv_edges" regulates the handling of edges with same pair of u-v nodes but different geometries.
When true keeps a center line between the two segments, unless one of the two segments is significantly longer than 
the other (>30%). In this case, the longer segment is deleted.
"""
nodes_graph, edges_graph = up.clean_network(nodes_graph, edges_graph, dead_ends = True, remove_disconnected_islands = True,
                            self_loops = True, same_uv_edges = True)

KeyboardInterrupt: 

In [ ]:
up.plot_lines(edges_graph, scheme = None,
              black_background = False, fig_size = 15, title = city_name+': Street Network')

In [ ]:
# obtaining graph from the case-study area 
graph = up.graph_fromGDF(nodes_graph, edges_graph)

In [ ]:
# Creating the dual geodataframes and the dual graph.
nodesDual_graph, edgesDual_graph = up.dual_gdf(nodes_graph, edges_graph, epsg)
dual_graph = up.dual_graph_fromGDF(nodesDual_graph, edgesDual_graph)

In [ ]:
"""
Different weights are used to extract the partitions. "None" indicates that no weights will be used 
(only topological relationships will matter). The function returns a GeoDataFrame with partitions assigned to edges,
with column named as "p_name_weight" (e.g. "p_length")
 
"""
weights = ['length', 'rad', None]
for i in weights:
    edges_graph = up.identify_regions(dual_graph, edges_graph, weight = i)

In [ ]:
# visualising
up.plot_lines(edges_graph, column = "p_no_weight", title = 'Districts', lw = 1.5, cmap = 'tab20', black_background = True, 
              legend = False, fig_size = 15)

In [ ]:
# polygonise districts

districts = edges_graph['p_rad']. 

# Barriers

In [ ]:
convex_hull = edges_graph.geometry.unary_union.convex_hull
waterways_barriers = up.waterways_barriers(place, distance, convex_hull)
railways = up.railways_barriers(place, distance, convex_hull)

In [ ]:
railways.to_file('railways2.shp', driver='ESRI Shapefile')

## Assign district to nodes and edges

In [ ]:
nodes_graph['district'] = 0
index_geo = nodes.columns.get_loc("geometry")+1  
spatial_index = edges.sindex # spatial index

for row in nodes.itertuples():
    point = row[index_geo]
    n = point.buffer(20)
    possible_matches_index = list(spatial_index.intersection(n.bounds))
    pm = edges.iloc[possible_matches_index].copy()
    dist = uf.dist_to_gdf(point, pm)
    
    district = edges.loc[dist[1]]['district'] 
    nodes.at[row[0], 'district'] = district
nodes['district'] = nodes.district.astype(int)

In [ ]:
edges['district'] = 0
index_u = edges.columns.get_loc("u")+1  
index_v = edges.columns.get_loc("v")+1  
# index_district = edges.columns.get_loc("district")+1  

for i in edges.itertuples():
    district_u = nodes.loc[i[index_u]].district
    district_v = nodes.loc[i[index_v]].district
#     if district_u == district_v: 
    edges.set_value(i[0], 'district', district_u) # then you may want to correct that manually

In [ ]:
nodes.to_file(IoC_output+'_nodes.shp', driver='ESRI Shapefile')
edges.to_file(IoC_output+'_paths.shp', driver='ESRI Shapefile')

In [ ]:

fig, ax = plt.subplots(1, figsize=(15,15))
plt.axis('equal')
ax.set_axis_off()
districts_polygonized.plot(ax = ax, column = 'district', cmap = 'Set2', alpha = 0.6)